# Task description
- Classify the speakers of given features.
- Main goal: Learn how to use transformer.
- Baselines:
  - Easy: Run sample code and know how to use transformer.
  - Medium: Know how to adjust parameters of transformer.
  - Strong: Construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer.
  - Boss: Implement [Self-Attention Pooling](https://arxiv.org/pdf/2008.01077v1.pdf) & [Additive Margin Softmax](https://arxiv.org/pdf/1801.05599.pdf) to further boost the performance.

- Other links
  - Kaggle: [link](https://www.kaggle.com/t/ac77388c90204a4c8daebeddd40ff916)
  - Slide: [link](https://docs.google.com/presentation/d/1HLAj7UUIjZOycDe7DaVLSwJfXVd3bXPOyzSb6Zk3hYU/edit?usp=sharing)
  - Data: [link](https://drive.google.com/drive/folders/1vI1kuLB-q1VilIftiwnPOCAeOOFfBZge?usp=sharing)

# Download dataset
- Data is [here](https://drive.google.com/drive/folders/1vI1kuLB-q1VilIftiwnPOCAeOOFfBZge?usp=sharing)

调Transformer参数+ConformerBlock: 0.773 0.77525

In [1]:
'''!wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partaa
!wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partab
!wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partac
!wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partad
!wget https://drive.google.com/drive/folders/1vI1kuLB-q1VilIftiwnPOCAeOOFfBZge
!cat Dataset.tar.gz.part* > Dataset.tar.gz

# unzip the file
tar zxvf Dataset.tar.gz
# 安装 gdown（一次即可）
# 1) 把整个文件夹拉到当前目录的临时子目录
!gdown --folder "https://drive.google.com/drive/folders/1vI1kuLB-q1VilIftiwnPOCAeOOFfBZge" -O ./_dl_tmp

# 2) 找到其中的 Dataset.tar.gz 并复制到当前目录
!cp "$(find ./_dl_tmp -type f -name 'Dataset.tar.gz' | head -n 1)" ./Dataset.tar.gz

# 3) 解压到当前目录（等价于你写的 tar zxvf Dataset.tar.gz，只是少打印点日志）
!tar -xzf ./Dataset.tar.gz -C .

# 4)（可选）确认解压结果
!ls -lah | head -n 50'''
from google.colab import drive
drive.mount('/content/drive')

# 把“副本”复制到当前目录并解压
!cp "/content/drive/MyDrive/CopyDataset.tar.gz" ./Dataset.tar.gz
!tar -xzf ./Dataset.tar.gz -C .
!ls -lah | head -n 30



Mounted at /content/drive
total 5.9G
drwxr-xr-x 1 root  root 4.0K Sep 13 11:19 .
drwxr-xr-x 1 root  root 4.0K Sep 13 11:13 ..
drwxr-xr-x 4 root  root 4.0K Sep  9 13:46 .config
drwxr-xr-x 2 69057  450 4.4M Mar 10  2022 Dataset
-rw------- 1 root  root 5.9G Sep 13 11:19 Dataset.tar.gz
drwx------ 5 root  root 4.0K Sep 13 11:17 drive
drwxr-xr-x 1 root  root 4.0K Sep  9 13:46 sample_data


## Fix Random Seed

In [2]:
import numpy as np
import torch
import random
#随机数跟作业三一样
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(87)

# Data

## Dataset
- Original dataset is [Voxceleb2](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox2.html).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb2.
- We randomly select 600 speakers from Voxceleb2.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training.
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary.
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [3]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

#处理数据集
class myDataset(Dataset):

	def __init__(self, data_dir, segment_len=128):#每次取128帧的时间片
		self.data_dir = data_dir
		self.segment_len = segment_len#一帧时长=特征时长/采样率

		# Load the mapping from speaker neme to their corresponding id.获得说话人的id
		mapping_path = Path(data_dir) / "mapping.json" #拼接成完整的路径
		mapping = json.load(mapping_path.open())#打开获得json内容
		self.speaker2id = mapping["speaker2id"]#从中提取spk

		# Load metadata of training data.#获得spk的特征
		metadata_path = Path(data_dir) / "metadata.json"
		metadata = json.load(open(metadata_path))["speakers"]#将json内容解析为python对象并取出speakers对应对的数据

		# Get the total number of speaker.
		self.speaker_num = len(metadata.keys())#获得数量 .keys是返回字典的所有键
		self.data = []
		for speaker in metadata.keys():
			for utterances in metadata[speaker]:#遍历说话人的所有语音条目
				self.data.append([utterances["feature_path"], self.speaker2id[speaker]])#构建【语音特征文件，id】的格式

	def __len__(self):
			return len(self.data)

	def __getitem__(self, index):
		feat_path, speaker = self.data[index]
		# Load preprocessed mel-spectrogram.
		mel = torch.load(os.path.join(self.data_dir, feat_path))#将.pt读成tensor 处理语音特征

		# Segmemt mel-spectrogram into "segment_len" frames.
		if len(mel) > self.segment_len:#segment_len=128
			# Randomly get the starting point of the segment.
			start = random.randint(0, len(mel) - self.segment_len)#随机找一个起点
			# Get a segment with "segment_len" frames.
			mel = torch.FloatTensor(mel[start:start+self.segment_len])#裁剪出128帧
		else:
			mel = torch.FloatTensor(mel)
		# Turn the speaker id into long for computing loss later.
		speaker = torch.FloatTensor([speaker]).long()#返回原长度
		return mel, speaker

	def get_speaker_number(self):
		return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.

In [4]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence

#将一个batch样本拼成张量
def collate_batch(batch):
	# Process features within a batch.
	"""Collate a batch of data."""
	mel, speaker = zip(*batch)#这个zip是按位置重新打包，输出是tuple类型
	'''
	batch = [
    (mel1, 0),
    (mel2, 1),
    (mel3, 2),
]
zip((mel1, 0), (mel2, 1), (mel3, 2)) 再把mel、speaker分别放到一起
	'''
	# Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
	#补齐128 用-20补 因为-20是静音帧
	mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
	# mel: (batch size, length, 40)
	return mel, torch.FloatTensor(speaker).long()#将tuple转换为pytorch再变long类型的


def get_dataloader(data_dir, batch_size, n_workers):
	"""Generate dataloader"""
	dataset = myDataset(data_dir)
	speaker_num = dataset.get_speaker_number()
	# Split dataset into training dataset and validation dataset
	#拆分训练集和验证集
	trainlen = int(0.9 * len(dataset))
	lengths = [trainlen, len(dataset) - trainlen]
	trainset, validset = random_split(dataset, lengths)

	train_loader = DataLoader(
		trainset,
		batch_size=batch_size,
		shuffle=True,
		drop_last=True,
		num_workers=n_workers,
		pin_memory=True,
		collate_fn=collate_batch,
	)
	valid_loader = DataLoader(
		validset,
		batch_size=batch_size,
		num_workers=n_workers,
		drop_last=True,
		pin_memory=True,
		collate_fn=collate_batch,
	)

	return train_loader, valid_loader, speaker_num

# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [5]:
!pip -q install conformer


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from conformer import ConformerBlock
import copy

class AMSoftmaxHead(nn.Module):
    """
    Additive Margin Softmax head (a.k.a. CosFace-style):
    - L2 normalize feature x and weight W
    - compute cosine logits: cos(theta) = x̂ · Ŵ
    - output scaled cosine: s * cos(theta)
    """
    def __init__(self, in_dim: int, n_classes: int, s: float = 30.0):
        super().__init__()
        self.W = nn.Parameter(torch.empty(n_classes, in_dim))
        nn.init.xavier_uniform_(self.W)
        self.s = s  # scaling factor

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: [B, D]; W: [C, D]
        x_norm = F.normalize(x, dim=-1)              # [B, D]
        W_norm = F.normalize(self.W, dim=-1)         # [C, D]
        cos = torch.matmul(x_norm, W_norm.t())       # [B, C]
        return self.s * cos                           # scaled cosine logits


class AMSoftmaxLoss(nn.Module):
    """
    Cross-Entropy on (s * (cos - m) for target class, cos for others)
    """
    def __init__(self, m: float = 0.20, s: float = 30.0):
        super().__init__()
        self.m = m
        self.s = s

    def forward(self, cosine_scaled: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        # cosine_scaled = s * cos(theta)
        # We subtract margin in the target logit BEFORE softmax.
        logits = cosine_scaled.clone()
        # subtract s*m (because logits already have s)
        logits[torch.arange(logits.size(0)), target] -= self.s * self.m
        return F.cross_entropy(logits, target)

class SAP(nn.Module):
    def __init__(self, in_dim, hidden=128):
        super().__init__()
        self.attn = nn.Sequential(
            nn.Linear(in_dim, hidden), nn.Tanh(),
            nn.Linear(hidden, 1, bias=False),
        )
    def forward(self, x, mask=None):          # x: [B,T,D]
        e = self.attn(x).squeeze(-1)          # [B,T]
        if mask is not None:
            e = e.masked_fill(~mask, -1e9)
        w = e.softmax(dim=1)                  # [B,T]
        return torch.bmm(w.unsqueeze(1), x).squeeze(1)  # [B,D]

class Classifier(nn.Module):
    # d_model: 编码维度；n_spks: 说话人数；n_layers: Conformer层数；n_mels: 梅尔维
    def __init__(self, d_model=224, n_spks=600, dropout=0.2):
        super().__init__()
        self.prenet = nn.Linear(40, d_model)
	#输入投影层
		# Project the dimension of features from that of input into d_model.
		#self.prenet = nn.Linear(40, d_model)#将mel维度升为80
		# TODO:
		#   Change Transformer to Conformer.
		#   https://arxiv.org/abs/2005.08100
		#Transformer编码层
		#self.encoder_layer = nn.TransformerEncoderLayer(
			#d_model=d_model, dim_feedforward=d_model*2, nhead=2,dropout=dropout)
		#self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=3)
        block = ConformerBlock(
            dim=d_model,
            dim_head=4, heads=4, ff_mult=4,
            conv_expansion_factor=2, conv_kernel_size=20,
            attn_dropout=dropout, ff_dropout=dropout, conv_dropout=dropout,
        )
        self.encoder = nn.Sequential(*[copy.deepcopy(block) for _ in range(3)])

        self.pool = SAP(d_model)
        '''
        self.pred_layer = nn.Sequential(
            nn.BatchNorm1d(d_model),
            nn.Linear(d_model, n_spks),
        )'''
        self.pred_layer = AMSoftmaxHead(in_dim=d_model, n_classes=n_spks, s=30.0)
    def forward(self, mels):
      # out: (batch size, length, d_model)
     out=self.prenet(mels)
     # out: (length, batch size, d_model)PyTorch 的 nn.TransformerEncoderLayer 要求输入维度是 (time, batch, feature)。
		#out = out.permute(1, 0, 2)
		# The encoder layer expect features in the shape of (length, batch size, d_model).
		#SAP是batchfirst 不用重排了
     out = self.encoder(out)
     # out: (batch size, length, d_model)
		#out = out.transpose(0, 1)
		# mean pooling
		#stats = out.mean(dim=1)
		#用SAP
     stats = self.pool(out)
     # out: (batch, n_spks) 分类层
     out = self.pred_layer(stats)
     return out


# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [7]:
#Warmup+余弦退火

import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
	optimizer: Optimizer,
	num_warmup_steps: int,
	num_training_steps: int,
	num_cycles: float = 0.5,
	last_epoch: int = -1,
):
	"""
	Create a schedule with a learning rate that decreases following the values of the cosine function between the
	initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
	initial lr set in the optimizer.

	Args:
		optimizer (:class:`~torch.optim.Optimizer`):
		The optimizer for which to schedule the learning rate.
		num_warmup_steps (:obj:`int`):
		The number of steps for the warmup phase.
		num_training_steps (:obj:`int`):
		The total number of training steps.
		num_cycles (:obj:`float`, `optional`, defaults to 0.5):
		The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
		following a half-cosine).
		last_epoch (:obj:`int`, `optional`, defaults to -1):
		The index of the last epoch when resuming training.

	Return:
		:obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
	"""
	def lr_lambda(current_step):
		# Warmup
		if current_step < num_warmup_steps:
			return float(current_step) / float(max(1, num_warmup_steps))
		# decadence余弦退火
		progress = float(current_step - num_warmup_steps) / float(
			max(1, num_training_steps - num_warmup_steps)
		)
		return max(
			0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
		)

	return LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [8]:
import torch


def model_fn(batch, model, criterion, device):
	"""Forward a batch through the model."""

	mels, labels = batch

	mels = mels.to(device)
	labels = labels.to(device)

	outs = model(mels)#model继承的nn.Module的

	loss = criterion(outs, labels)

	# Get the speaker id with highest probability.
	preds = outs.argmax(1)
	# Compute accuracy.
	accuracy = torch.mean((preds == labels).float())

	return loss, accuracy

# Validate
- Calculate accuracy of the validation set.

In [9]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device):
	"""Validate on validation set."""

	model.eval()
	running_loss = 0.0
	running_accuracy = 0.0
	pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

	for i, batch in enumerate(dataloader):
		with torch.no_grad():
			loss, accuracy = model_fn(batch, model, criterion, device)
			running_loss += loss.item()
			running_accuracy += accuracy.item()

		pbar.update(dataloader.batch_size)
		pbar.set_postfix(
			loss=f"{running_loss / (i+1):.2f}",
			accuracy=f"{running_accuracy / (i+1):.2f}",
		)

	pbar.close()
	model.train()

	return running_accuracy / len(dataloader)

# Main function

In [10]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset",
		"save_path": "model.ckpt",
		"batch_size": 32,
		"n_workers": 8,
		"valid_steps": 2000,
		"warmup_steps": 1000,
		"save_steps": 10000,
		"total_steps": 70000,
	}

	return config


def main(
	data_dir,
	save_path,
	batch_size,
	n_workers,
	valid_steps,
	warmup_steps,
	total_steps,
	save_steps,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
	train_iterator = iter(train_loader)
	print(f"[Info]: Finish loading data!",flush = True)

	model = Classifier(n_spks=speaker_num).to(device)
	criterion = nn.CrossEntropyLoss()
	optimizer = AdamW(model.parameters(), lr=1e-3)
	scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
	print(f"[Info]: Finish creating model!",flush = True)



	best_accuracy = -1.0
	best_state_dict = None

	pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

	for step in range(total_steps):
		# Get data
		try:
			batch = next(train_iterator)
		except StopIteration:
			train_iterator = iter(train_loader)
			batch = next(train_iterator)

		loss, accuracy = model_fn(batch, model, criterion, device)
		batch_loss = loss.item()
		batch_accuracy = accuracy.item()

		# Updata model
		loss.backward()
		optimizer.step()
		scheduler.step()
		optimizer.zero_grad()

		# Log
		pbar.update()
		pbar.set_postfix(
			loss=f"{batch_loss:.2f}",
			accuracy=f"{batch_accuracy:.2f}",
			step=step + 1,
		)

		# Do validation
		if (step + 1) % valid_steps == 0:
			pbar.close()

			valid_accuracy = valid(valid_loader, model, criterion, device)

			# keep the best model
			if valid_accuracy > best_accuracy:
				best_accuracy = valid_accuracy
				best_state_dict = model.state_dict()

			pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

		# Save the best model so far.
		if (step + 1) % save_steps == 0 and best_state_dict is not None:
			torch.save(best_state_dict, save_path)
			pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

	pbar.close()


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [02:00<00:00, 16.58 step/s, accuracy=0.34, loss=2.90, step=2000]
Valid: 100% 5664/5667 [00:06<00:00, 822.70 uttr/s, accuracy=0.26, loss=3.47] 
Train: 100% 2000/2000 [01:54<00:00, 17.52 step/s, accuracy=0.41, loss=1.99, step=4000]
Valid: 100% 5664/5667 [00:06<00:00, 904.91 uttr/s, accuracy=0.49, loss=2.24] 
Train: 100% 2000/2000 [01:56<00:00, 17.20 step/s, accuracy=0.47, loss=1.78, step=6000]
Valid: 100% 5664/5667 [00:06<00:00, 851.16 uttr/s, accuracy=0.61, loss=1.65] 
Train: 100% 2000/2000 [01:54<00:00, 17.45 step/s, accuracy=0.78, loss=0.89, step=8000]
Valid: 100% 5664/5667 [00:04<00:00, 1291.76 uttr/s, accuracy=0.68, loss=1.34]
Train: 100% 2000/2000 [01:52<00:00, 17.80 step/s, accuracy=0.88, loss=0.59, step=1e+4]
Valid: 100% 5664/5667 [00:05<00:00, 1079.03 uttr/s, accuracy=0.74, loss=1.11]
Train:   0% 3/2000 [00:00<02:25, 13.68 step/s, accuracy=0.75, loss=0.94, step=1e+4]

Step 10000, best model saved. (accuracy=0.7373)


...

Step 50000, best model saved. (accuracy=0.9227)


Train: 100% 2000/2000 [01:52<00:00, 17.80 step/s, accuracy=1.00, loss=0.06, step=52000]
Valid: 100% 5664/5667 [00:04<00:00, 1259.25 uttr/s, accuracy=0.93, loss=0.32]
Train: 100% 2000/2000 [01:53<00:00, 17.70 step/s, accuracy=0.97, loss=0.08, step=54000]
Valid: 100% 5664/5667 [00:05<00:00, 1083.19 uttr/s, accuracy=0.92, loss=0.35]
Train: 100% 2000/2000 [01:53<00:00, 17.56 step/s, accuracy=1.00, loss=0.04, step=56000]
Valid: 100% 5664/5667 [00:05<00:00, 1131.52 uttr/s, accuracy=0.93, loss=0.33]
Train: 100% 2000/2000 [01:52<00:00, 17.75 step/s, accuracy=1.00, loss=0.03, step=58000]
Valid: 100% 5664/5667 [00:04<00:00, 1292.20 uttr/s, accuracy=0.93, loss=0.32]
Train: 100% 2000/2000 [01:53<00:00, 17.67 step/s, accuracy=1.00, loss=0.04, step=6e+4]
Valid: 100% 5664/5667 [00:04<00:00, 1316.60 uttr/s, accuracy=0.93, loss=0.31]
Train:   0% 4/2000 [00:00<01:55, 17.31 step/s, accuracy=1.00, loss=0.04, step=6e+4]

Step 60000, best model saved. (accuracy=0.9322)


Train: 100% 2000/2000 [01:53<00:00, 17.63 step/s, accuracy=1.00, loss=0.02, step=62000]
Valid: 100% 5664/5667 [00:04<00:00, 1173.14 uttr/s, accuracy=0.93, loss=0.31]
Train: 100% 2000/2000 [01:52<00:00, 17.85 step/s, accuracy=1.00, loss=0.04, step=64000]
Valid: 100% 5664/5667 [00:06<00:00, 812.89 uttr/s, accuracy=0.93, loss=0.30] 
Train: 100% 2000/2000 [01:51<00:00, 17.92 step/s, accuracy=1.00, loss=0.03, step=66000]
Valid: 100% 5664/5667 [00:04<00:00, 1241.39 uttr/s, accuracy=0.93, loss=0.31]
Train: 100% 2000/2000 [01:52<00:00, 17.75 step/s, accuracy=1.00, loss=0.04, step=68000]
Valid: 100% 5664/5667 [00:04<00:00, 1315.32 uttr/s, accuracy=0.93, loss=0.32]
Train: 100% 2000/2000 [01:52<00:00, 17.80 step/s, accuracy=1.00, loss=0.02, step=7e+4]
Valid: 100% 5664/5667 [00:04<00:00, 1270.22 uttr/s, accuracy=0.93, loss=0.30]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 70000, best model saved. (accuracy=0.9322)


# Inference

## Dataset of inference

In [11]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
	def __init__(self, data_dir):
		testdata_path = Path(data_dir) / "testdata.json"
		metadata = json.load(testdata_path.open())
		self.data_dir = data_dir
		self.data = metadata["utterances"]

	def __len__(self):
		return len(self.data)

	def __getitem__(self, index):
		utterance = self.data[index]
		feat_path = utterance["feature_path"]
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		return feat_path, mel


def inference_collate_batch(batch):
	"""Collate a batch of data."""
	feat_paths, mels = zip(*batch)

	return feat_paths, torch.stack(mels)

## Main funcrion of Inference

In [12]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset",
		"model_path": "./model.ckpt",
		"output_path": "./output.csv",
	}

	return config


def main(
	data_dir,
	model_path,
	output_path,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	mapping_path = Path(data_dir) / "mapping.json"
	mapping = json.load(mapping_path.open())

	dataset = InferenceDataset(data_dir)
	dataloader = DataLoader(
		dataset,
		batch_size=1,
		shuffle=False,
		drop_last=False,
		num_workers=8,
		collate_fn=inference_collate_batch,
	)
	print(f"[Info]: Finish loading data!",flush = True)

	speaker_num = len(mapping["id2speaker"])
	model = Classifier(n_spks=speaker_num).to(device)
	model.load_state_dict(torch.load(model_path))
	model.eval()
	print(f"[Info]: Finish creating model!",flush = True)

	results = [["Id", "Category"]]
	for feat_paths, mels in tqdm(dataloader):
		with torch.no_grad():
			mels = mels.to(device)
			outs = model(mels)
			preds = outs.argmax(1).cpu().numpy()
			for feat_path, pred in zip(feat_paths, preds):
				results.append([feat_path, mapping["id2speaker"][str(pred)]])

	with open(output_path, 'w', newline='') as csvfile:
		writer = csv.writer(csvfile)
		writer.writerows(results)


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


  0%|          | 0/8000 [00:00<?, ?it/s]